# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [1]:
using LinearAlgebra, Test

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.


In [2]:
using ColorBitstring

function mul(A, x) #column by column multiplication: x1*a1 + ... + xn*an, for xi elements of x, ai columns of A
    m,n = size(A)
    c = zeros(eltype(x), m) # eltype is the type of the elements of a vector/matrix
    for j = 1:n, k = 1:m
        c[k] += A[k, j] * x[j]
    end
    c
end


A = [1 1;
     5 2^-52]
x = [0.1,
     0.2]

A*x, mul(A,x)
y = (A*x)[1]
z = (mul(A,x))[1]
w = (A*x)[2]
v = (mul(A,x))[2]
printlnbits(y) == printlnbits(z), printlnbits(w) == printlnbits(v)


0011111111010011001100110011001100110011001100110011001100110100
0011111111010011001100110011001100110011001100110011001100110100
0011111111100000000000000000000000000000000000000000000000000000
0011111111100000000000000000000000000000000000000000000000000000


(true, true)

## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [19]:
function ldiv(U::UpperTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using back-substitution
    for k = n:-1:1
        for j = k+1:n
            x[k] = x[k] - (U[k, j]*x[j])
        end
        x[k] = (x[k] + b[k])/U[k, k]
    end
    x
end

function ldiv(U::LowerTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using forward-substitution
    for k = 1:n
        for j = 1:k-1
            x[k] = x[k] - (U[k, j]*x[j])
        end
        x[k] = (x[k] + b[k])/U[k, k]
    end
    x
end

A = UpperTriangular([1 1; 0 1])
b = [1, 3]

B = LowerTriangular([1 0 0; 1 1 0; 1 1 1])
c = [1, 1, 1]

ldiv(B,c) == B \ c, ldiv(A,b) == A \ b

(true, true)

**Problem 2.2⋆** Given $𝐱 ∈ ℝ^n$, find a lower triangular matrix of the form
$$
L = I - 2 𝐯 𝐞_1^⊤
$$
such that:
$$
L 𝐱 = x_1 𝐞_1.
$$
What does $L𝐲$ equal if $𝐲  ∈ ℝ^n$ satisfies $y_1 = 𝐞_1^⊤ 𝐲 = 0$?

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [29]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]
    if k == j
        ret = d[k]
    elseif k == j-1
        ret = du[k]
    elseif k == j-2
        ret = du2[k]
    else
        ret = 0
    end
    ret
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end

    # TODO: modify d,du,du2 so that U[k,j] == v
    if k == j
        d[k] = v
    elseif k == j-1
        du[k] = v
    elseif k == j-2
        du2[k] = v
    end
    U # by convention we return the matrix
end

#U = [1 2 3 0 0; 0 1 2 3 0; 0 0 1 2 3; 0 0 0 1 2; 0 0 0 0 1]
d = Float64[1,1,1,1,1]
du = Float64[2,2,2,2]
du2 = Float64[3,3,3]

U = UpperTridiagonal(d,du,du2)

5

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [38]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    # TODO: populate b so that U*x == b (up to rounding)
    n = size(U,1)
    for k = 1:n-2
        b[k] = U.d[k]*x[k] + U.du[k]*x[k+1] + U.du2[k]*x[k+2]
    end
    b[n-1] = U.d[n-1]*x[n-1] + U.du[n-1]*x[n]
    b[n] = U.d[n]*x[n]

    b
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    # TODO: populate x so that U*x == b (up to rounding)
    n = size(U,1)
    x[n] = b[n]/d[n]
    x[n-1] = (b[n-1] - du[n-1]*x[n])/d[n-1]
    for k = n-2:-1:1
        x[k] = (b[k] - du[k]*x[k+1] - du2[k]*x[k+2])/d[k]
    end
    x
end

M = [1 2 3 0 0; 0 1 2 3 0; 0 0 1 2 3; 0 0 0 1 2; 0 0 0 0 1] #AbstractMatrix type of U (upper tridiagonal)
x = [1,2,3,4,5]
b = U*x

U*x ≈ M*x, U \ b ≈ M \ b, x ≈ U \ b

(true, true, true)

## 4. Permutations

**Problem 4.1⋆** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$


**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [39]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    #P[k,j] = ek' P ej = delta(sigma(k),j)
    #sigma(k) = p[k]
    #access p via P.p
    P.p[k] == j ? 1 : 0 #return 1 if true, 0 if false

end

function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    # apply sigma to x: sigma = P.p, notation for sigma(x) = x[sigma]
    x[P.p]
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

Test Passed
  Expression: P == (I(5))[p, :]
   Evaluated: [1 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 1; 0 0 … 0 0] == sparse([1, 3, 5, 2, 4], [1, 2, 3, 4, 5], Bool[1, 1, 1, 1, 1], 5, 5)

## 5. Orthogonal matrices

**Problem 5.1⋆** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q 𝐯\| = \|𝐯\|.
$$


**Problem 5.2⋆** Show that the eigenvalues $λ$ of an orthogonal matrix $Q$ are
on the unit circle: $|λ| = 1$.


**Problem 5.3⋆** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [40]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    # TODO: Return Q[k,j]
    #Q[k, j] = (I - 2vv')[k,j] = delta(k,j) - 2v[k]v[j]
    (k==j) - 2*Q.v[k]*Q.v[j]
end

function *(P::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
    #v = Q.v
    x - 2*Q.v*((Q.v)'*x)
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I

Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [1.0000000000000002 2.220446049250313e-16 … -1.0581813203458523e-16 5.551115123125783e-17; 2.220446049250313e-16 1.0000000000000002 … -2.1510571102112408e-16 2.7755575615628914e-17; … ; -1.0581813203458523e-16 -2.1510571102112408e-16 … 1.0 -2.7755575615628914e-17; 5.551115123125783e-17 2.7755575615628914e-17 … -2.7755575615628914e-17 1.0] ≈ UniformScaling{Bool}(true)

**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass.

In [42]:
function householderreflection(s::Bool, x::AbstractVector)
    # TODO: implement Householder reflection, returning a Reflection
    if s == false
        y = norm(x)*[1; zeros(length(x)-1)] + x
    else
        y = -norm(x)*[1; zeros(length(x)-1)] + x
    end
    w = y/norm(y)
    I - 2w*w'
end

x = randn(5)
print(x)
Q = householderreflection(true, x)
#@test Q isa Reflection
@test Q*x ≈ norm(x)*[1; zeros(length(x)-1)]

Q̃ = householderreflection(false, x)
#@test Q̃ isa Reflection
@test Q̃*x ≈ -norm(x)*[1; zeros(length(x)-1)]

[-1.431513195041556, 1.5037230635726384, 0.3534631526912122, 2.1609706072256647, 0.43976543592769035]

Test Passed
  Expression: Q̃ * x ≈ -(norm(x)) * [1; zeros(length(x) - 1)]
   Evaluated: [-3.049350239598381, -3.3306690738754696e-16, -1.700029006457271e-16, -5.828670879282072e-16, -1.3059788590865346e-16] ≈ [-3.049350239598381, -0.0, -0.0, -0.0, -0.0]

0